**Predicting price trends of digital products using various forecasting techniques: On the example of the Steam Community Marketplace**

Research questions: 
- Are prices predictible at all?
- Are machine learning methods better in predicting prices for digital goods compared to traditional methods (ARIMA,…)?
- What is the most precise algorithm for predicting prices of digital goods?


Loading packages

In [ ]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import time 
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import feather
import datetime 

Set cookie

In [ ]:
cookie = {'steamLoginSecure': 'xxx'}; #cookie for the steam marketplace obtained through the browser

Game_id='730' #game id for a specific game

Getting amount and list of all items per game

In [ ]:
Game_id='730' #game id for a specific game
item_list_all=[]


items_count = requests.get('https://steamcommunity.com/market/search/render/?search_descriptions=0&sort_column=default&sort_dir=desc&appid='+Game_id+'&norender=1&count=100', cookies=cookie).json();
items_count = items_count['total_count'] #total number of ingame items for this game
print(items_count)

#steam only allows for 100 items per time and needs some time between requests
for item_rank in range(0,items_count+50,50): #get 100 item batches per loop

          time.sleep(random.uniform(1, 2.5)) #random wait time between api calls
          while True:
            try:
              item_list = requests.get('https://steamcommunity.com/market/search/render/?start='+str(item_rank)+'&count=100&search_descriptions=0&sort_column=default&sort_dir=desc&appid='+Game_id+'&norender=1&count=5000', cookies=cookie).json();
              print(item_list)
              item_list = item_list['results']
              if not item_list and (item_rank < items_count) : #if the item_list that we got as a response is empty and it shouldn't be empty (item rank is lower than total items)
                print('empty list')
                time.sleep(random.uniform(1, 2.5))
                continue
              else:
                for item in item_list: 
                  item_list_all.append(item['hash_name']) #save all items to a list
            except TypeError:
              print('TypeError')
              time.sleep(random.uniform(1, 2.5))
              continue
            break
 
            
        
item_list_all = [*set(item_list_all)] #remove duplicates which can occure if list is reshuffled while data is obtained

Check if every item is in the list

In [ ]:
print(len(item_list_all))

17327


Save items to txt file so the item list doesn't have to collected everytime 




In [ ]:
#save item_list_all as a txt file
with open('item_list_all_new4.txt', 'w') as f:
    for line in item_list_all:
        f.write("%s\n" % line)

**Break down item list in case you get disconnected from the api so not everything is lost**

In [ ]:
with open("item_list_all_new4.txt","r") as f:
 print(len(list(f)))

17327


In [ ]:
data_all = pd.DataFrame()
n=1000 #blocks auf 1000 
with open("item_list_all_new4.txt","r") as f:
    item_list=list(f)
    for i in range(0, len(item_list), n):
    
        #data_all = pd.DataFrame()
        for item in item_list[i:i + n]:
          
          time.sleep(random.uniform(0.2, 1))
          print(item)
          item_name = item.replace(' ','%20'); # for http search requests spaces need to be converted to '%20'
          item_name = item_name.replace('&','%26'); # also '&' need to be converted to '%26'
          while True:
                try:
                  price_data = requests.get('https://steamcommunity.com/market/pricehistory/?appid='+Game_id+'&market_hash_name='+item_name, cookies=cookie).json(); # get item data
                  print(price_data)
                  df = json_normalize(price_data, 'prices') #convert json format to DataFrame
                  if df.empty: #if the item has no price history then skip it
                    print('no price history')
                    #print(df)
                    pass
                  else:
                    df = df.set_axis(['Date', 'Average_Price', 'Volume'], axis=1, inplace=False) # set names for columns
                    df['Volume']=df['Volume'].astype('int')
                    df['Item'] =item
                    #print(df)
                    data_all=data_all.append(df)
                except TypeError:
                  print('TypeError')
                  pass
                break
          #print(data_all)
        data_feather=data_all
        data_feather=data_feather.reset_index()
        data_feather.to_feather(str(i)+"_block.feather")
        print(data_all)
        

**save to feather file**

In [ ]:
data_feather=data_new
data_feather=data_feather.reset_index()
data_feather.to_feather("data.feather")

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/data_final.feather drive/MyDrive/Master/data_final.feather

In [ ]:
!cp drive/MyDrive/Master/data_date_clean.feather /content/data_date_clean.feather

In [ ]:
n=1000000 #blocks auf 1000
data_new = pd.DataFrame()
df=data_load
for i in range(0, len(data_load), n):
  print(i)
  print(i+n)	
  df_filter=df[i:i+n] 
  df_filter['Date'] = df_filter['Date'].str[:-7] #remove unwanted parts in the 'Date' column
  df_filter['Date'] = pd.to_datetime(df_filter['Date']) #convert 'Date' column to datetime format
  #print(df_filter)
  data_new=data_new.append(df_filter)
data_new=data_new.reset_index(drop=True)
data_new.to_feather("data_date_clean.feather")

In [ ]:
#Playerbase statistic from steamcharts.com
player_data = requests.get('https://steamcharts.com/app/730/chart-data.json').json(); # get player number history 
df_player = pd.DataFrame(player_data)
df_player = df_player.set_axis(['Date', 'Player_Count'], axis=1, inplace=False)
df_player['Date'] = pd.to_datetime(df_player['Date'], unit='ms').dt.date
df_player=df_player[df_player['Date']< datetime.date(2022,9,1)]
df_player=df_player.reset_index(drop=True)
df_player.to_feather("player_count.feather")
print(df_player.to_string())